In [1]:
import pandas as pd
import numpy as np
import yfinance as yf



In [3]:
ticker = "AAPL"
start_date = "2020-01-01"
end_date ="2022-01-01"


stock_data = yf.download(ticker, start=start_date, end=end_date)




[*********************100%%**********************]  1 of 1 completed


In [4]:
stock_data.to_pickle('test.pkl')


In [5]:
print(stock_data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 505 entries, 2020-01-02 to 2021-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       505 non-null    float64
 1   High       505 non-null    float64
 2   Low        505 non-null    float64
 3   Close      505 non-null    float64
 4   Adj Close  505 non-null    float64
 5   Volume     505 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 27.6 KB
None


In [6]:
print(stock_data.isnull().sum())

stock_data['Daily_return']



Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [7]:
# Calculate daily returns
stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change()

# Add moving averages
stock_data['MA_50'] = stock_data['Adj Close'].rolling(window=50).mean()
stock_data['MA_200'] = stock_data['Adj Close'].rolling(window=200).mean()

In [8]:
# Select relevant features
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Daily_Return', 'MA_50', 'MA_200']
input_data = stock_data[features]

# Normalize data if needed (optional)
normalized_data = (input_data - input_data.mean()) / input_data.std()

# Define sequence length (e.g., 30 days)
sequence_length = 30

# Create input sequences
X = []
y = []

for i in range(len(normalized_data) - sequence_length):
    X.append(normalized_data.iloc[i:i + sequence_length].values)
    y.append(normalized_data.iloc[i + sequence_length]['Close'])

X = np.array(X)
y = np.array(y)

split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Assuming 'X_train' and 'y_train' are your input features and target variables

num_features = 8

# Positional encoding layer
class PositionalEncoding(layers.Layer):
    def __init__(self, sequence_length, output_dim):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(sequence_length, output_dim)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

    def get_config(self):
        config = super(PositionalEncoding, self).get_config()
        return config

    def positional_encoding(self, sequence_length, output_dim):
        angle_rads = self.get_angles(np.arange(sequence_length)[:, np.newaxis], np.arange(output_dim)[np.newaxis, :], output_dim)
        sines = np.sin(angle_rads[:, 0::2])
        cosines = np.cos(angle_rads[:, 1::2])

        pos_encoding = np.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_angles(self, positions, i, d_model):
        angles = 1 / np.power(10000.0, (2 * (i // 2)) / np.float32(d_model))
        return positions * angles

def build_model(sequence_length, num_heads, ff_dim, dropout=0, mlp_units=[128]):
    inputs = keras.Input(shape=(sequence_length, num_features))  # Assuming num_features is the number of features in your input data
    x = PositionalEncoding(sequence_length, num_features)(inputs)
    
    for units in mlp_units:
        x = transformer_encoder(x, num_features, num_heads, ff_dim, dropout)
    
    outputs = layers.TimeDistributed(layers.Conv1D(filters=num_features, kernel_size=1, activation="linear"))(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


# Transformer model
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

# Build the Transformer model


# Define hyperparameters
sequence_length = 30
num_heads = 2
ff_dim = 30
dropout = 0.25
num_features = 8

# Build and compile the model
model = build_model(sequence_length, num_heads, ff_dim, dropout)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions
predictions = model.predict(X_test)


Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/mean_squared_error/BroadcastGradientArgs defined at (most recent call last):
  File "C:\Users\Rober\.pyenv\pyenv-win\versions\3.10.11\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\Rober\.pyenv\pyenv-win\versions\3.10.11\lib\runpy.py", line 86, in _run_code

  File "c:\Users\Rober\.virtualenvs\AASD4010-XSwstjEU\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\Rober\.virtualenvs\AASD4010-XSwstjEU\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Rober\.virtualenvs\AASD4010-XSwstjEU\lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "c:\Users\Rober\.virtualenvs\AASD4010-XSwstjEU\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\Rober\.pyenv\pyenv-win\versions\3.10.11\lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\Users\Rober\.pyenv\pyenv-